Import Libraries 


In [55]:
import pandas as pd


Load data and display first five rows

In [56]:
loan_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_train.csv" )
loan_data.head()

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,LP002305,Female,No,0,Graduate,No,4547,0.0,115.0,360.0,1.0,Semiurban,1
1,1,LP001715,Male,Yes,3+,Not Graduate,Yes,5703,0.0,130.0,360.0,1.0,Rural,1
2,2,LP002086,Female,Yes,0,Graduate,No,4333,2451.0,110.0,360.0,1.0,Urban,0
3,3,LP001136,Male,Yes,0,Not Graduate,Yes,4695,0.0,96.0,NaN,1.0,Urban,1
4,4,LP002529,Male,Yes,2,Graduate,No,6700,1750.0,230.0,300.0,1.0,Semiurban,1


Exploratory Data Analysis 


In [57]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         491 non-null    int64  
 1   Loan_ID            491 non-null    object 
 2   Gender             481 non-null    object 
 3   Married            490 non-null    object 
 4   Dependents         482 non-null    object 
 5   Education          491 non-null    object 
 6   Self_Employed      462 non-null    object 
 7   ApplicantIncome    491 non-null    int64  
 8   CoapplicantIncome  491 non-null    float64
 9   LoanAmount         475 non-null    float64
 10  Loan_Amount_Term   478 non-null    float64
 11  Credit_History     448 non-null    float64
 12  Property_Area      491 non-null    object 
 13  Loan_Status        491 non-null    int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 53.8+ KB


In [58]:
loan_data.isnull().sum()

Unnamed: 0            0
Loan_ID               0
Gender               10
Married               1
Dependents            9
Education             0
Self_Employed        29
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           16
Loan_Amount_Term     13
Credit_History       43
Property_Area         0
Loan_Status           0
dtype: int64

In [59]:
def impute_nan_most_frequent_category(DataFrame,ColName):
    # .mode()[0] - gives first category name
     most_frequent_category=DataFrame[ColName].mode()[0]
    
    # replace nan values with most occured category
     DataFrame[ColName] = DataFrame[ColName]
     DataFrame[ColName].fillna(most_frequent_category,inplace=True)

In [60]:
def impute_nan_most_frequent_numerical(DataFrame,ColName):
    # .mode()[0] - gives first category name
     most_frequent_category=DataFrame[ColName].mean()
    
    # replace nan values with most occured category
     DataFrame[ColName] = DataFrame[ColName]
     DataFrame[ColName].fillna(most_frequent_category,inplace=True)

In [61]:
for Columns in ['Gender','Married','Dependents','Self_Employed']:
    impute_nan_most_frequent_category(loan_data,Columns)

In [62]:
for Columns in ['LoanAmount','Loan_Amount_Term','Credit_History']:
    impute_nan_most_frequent_numerical(loan_data,Columns)

In [63]:
loan_data['Dependents'] = loan_data['Dependents'].replace('3+', '3')
loan_data['Dependents'] = loan_data['Dependents'].astype(str).astype(float)

In [65]:
loan_data = loan_data.drop(['Unnamed: 0', 'Loan_ID'], axis=1)

In [66]:
loan_data = pd.get_dummies(loan_data)

In [67]:
loan_data.head()


,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.0,4547,0.0,115.0,360.000000,1.0,1,1,0,1,0,1,0,1,0,0,1,0
1,3.0,5703,0.0,130.0,360.000000,1.0,1,0,1,0,1,0,1,0,1,1,0,0
2,0.0,4333,2451.0,110.0,360.000000,1.0,0,1,0,0,1,1,0,1,0,0,0,1
3,0.0,4695,0.0,96.0,341.297071,1.0,1,0,1,0,1,0,1,0,1,0,0,1
4,2.0,6700,1750.0,230.0,300.000000,1.0,1,0,1,0,1,1,0,1,0,0,1,0


In [82]:
y = loan_data['Loan_Status']
x = loan_data.drop(['Loan_Status'], axis = 1)

In [83]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Dependents               491 non-null    float64
 1   ApplicantIncome          491 non-null    int64  
 2   CoapplicantIncome        491 non-null    float64
 3   LoanAmount               491 non-null    float64
 4   Loan_Amount_Term         491 non-null    float64
 5   Credit_History           491 non-null    float64
 6   Gender_Female            491 non-null    uint8  
 7   Gender_Male              491 non-null    uint8  
 8   Married_No               491 non-null    uint8  
 9   Married_Yes              491 non-null    uint8  
 10  Education_Graduate       491 non-null    uint8  
 11  Education_Not Graduate   491 non-null    uint8  
 12  Self_Employed_No         491 non-null    uint8  
 13  Self_Employed_Yes        491 non-null    uint8  
 14  Property_Area_Rural      4

In [84]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=27)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [85]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [79]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [86]:
import xgboost as xgb
clf = xgb.XGBClassifier().fit(x_train, y_train)

In [87]:
y_train_pred = clf.predict(x_train)
from sklearn.metrics import accuracy_score,f1_score
#train
print(accuracy_score(y_train,y_train_pred)*100)
print(f1_score(y_train,y_train_pred)*100)
# test
y_test_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_test_pred)*100)
print(f1_score(y_test,y_test_pred)*100)

86.95652173913044
91.51943462897526
76.42276422764228
83.97790055248619


In [96]:
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_test.csv')

In [97]:
for Columns in ['Gender','Married','Dependents','Self_Employed']:
    impute_nan_most_frequent_category(test_data,Columns)

In [98]:
for Columns in ['LoanAmount','Loan_Amount_Term','Credit_History']:
    impute_nan_most_frequent_numerical(test_data,Columns)

In [99]:
test_data['Dependents'] = test_data['Dependents'].replace('3+', '3')
test_data['Dependents'] = test_data['Dependents'].astype(str).astype(float)

In [100]:
test_data = test_data.drop(['Loan_ID'], axis=1)

In [103]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Dependents               123 non-null    float64
 1   ApplicantIncome          123 non-null    int64  
 2   CoapplicantIncome        123 non-null    float64
 3   LoanAmount               123 non-null    float64
 4   Loan_Amount_Term         123 non-null    float64
 5   Credit_History           123 non-null    float64
 6   Gender_Female            123 non-null    uint8  
 7   Gender_Male              123 non-null    uint8  
 8   Married_No               123 non-null    uint8  
 9   Married_Yes              123 non-null    uint8  
 10  Education_Graduate       123 non-null    uint8  
 11  Education_Not Graduate   123 non-null    uint8  
 12  Self_Employed_No         123 non-null    uint8  
 13  Self_Employed_Yes        123 non-null    uint8  
 14  Property_Area_Rural      1

In [102]:
test_data = pd.get_dummies(test_data)

In [104]:
y_pred = clf.predict(test_data)
result_df = pd.DataFrame()
result_df['prediction'] = y_pred
result_df.reset_index(drop=True, inplace=True)
result_df.to_csv('sample_submission.csv', index = False)